# # 모듈
---

In [3]:
# !git clone https://github.com/Bharath2/Informed-RRT-star
# %cd /content/Informed-RRT-star
# %pip install -qr requirements.txt
!pip install rtree
!pip install pymysql
!pip install pyproj
!pip install geopandas
!pip install pyshp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.1 MB/s 
     |████████████████████████████████| 16.7 MB 39.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 89 kB/s 


In [7]:
gdopandas.__version__

NameError: ignored

In [ ]:
# 그리드 구성
import numpy as np
import sys
import re
import math
import matplotlib.pyplot as plt
import pymysql
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)

# # Map-to-Grid
---

In [ ]:
class map_grid():

    def __init__(self, grid = 5):
        import math

        self.NX = 149            ## X축 격자점 수
        self.NY = 253            ## Y축 격자점 수
        self.Re = 6371.00877     ##  지도반경
        self.grid = grid         ##  격자간격 (km)
        self.slat1 = 30.0        ##  표준위도 1
        self.slat2 = 60.0        ##  표준위도 2
        self.olon = 126.0        ##  기준점 경도
        self.olat = 38.0         ##  기준점 위도
        self.xo = 210 / self.grid     ##  기준점 X좌표
        self.yo = 675 / self.grid     ##  기준점 Y좌표
        self.first = 0
        self.math = math

        if self.first == 0 :
            self.PI = math.asin(1.0) * 2.0
            self.DEGRAD = self.PI/ 180.0
            self.RADDEG = 180.0 / self.PI

            self.re = self.Re / self.grid
            self.slat1 = self.slat1 * self.DEGRAD
            self.slat2 = self.slat2 * self.DEGRAD
            self.olon = self.olon * self.DEGRAD
            self.olat = self.olat * self.DEGRAD

            self.sn = math.tan(self.PI * 0.25 + self.slat2 * 0.5) / math.tan(self.PI * 0.25 + self.slat1 * 0.5)
            self.sn = math.log(math.cos(self.slat1) / math.cos(self.slat2)) / math.log(self.sn)
            self.sf = math.tan(self.PI * 0.25 + self.slat1 * 0.5)
            self.sf = math.pow(self.sf, self.sn) * math.cos(self.slat1) / self.sn
            self.ro = math.tan(self.PI * 0.25 + self.olat * 0.5)
            self.ro = self.re * self.sf / math.pow(self.ro, self.sn)
            self.first = 1

    def mapToGrid(self, lat, lon, code = 0):
        import math

        ra = math.tan(self.PI * 0.25 + lat * self.DEGRAD * 0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI :
            theta -= 2.0 * self.PI
        if theta < -self.PI :
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        x = int(x + 1.5)
        y = int(y + 1.5)
        return x, y

    def gridToMap(self, x, y, code = 1):
        import math

        x = x - 1
        y = y - 1
        xn = x - self.xo
        yn = self.ro - y + self.yo
        ra = math.sqrt(xn * xn + yn * yn)
        if self.sn < 0.0 :
            ra = -ra
        alat = math.pow((self.re * self.sf / ra), (1.0 / self.sn))
        alat = 2.0 * math.atan(alat) - self.PI * 0.5
        if math.fabs(xn) <= 0.0 :
            theta = 0.0
        else :
            if math.fabs(yn) <= 0.0 :
                theta = self.PI * 0.5
                if xn < 0.0 :
                    theta = -theta
            else :
                theta = math.atan2(xn, yn)
        alon = theta / self.sn + self.olon
        lat = alat * self.RADDEG
        lon = alon * self.RADDEG

        return lat, lon

mg1 = map_grid(grid = 1)
mg5 = map_grid(grid = 5)

# # 데이터 로드
---

## # 병원 데이터
---

In [ ]:
import pymysql
import pandas as pd

conn = pymysql.connect(host='34.64.132.212', user='root', password='ASewetsvc124~1242#%1wsdeWXV', db='source')
sql_state='SELECT * FROM `군집화된병원6`'
hospital=pd.read_sql_query(sql_state, conn)

hospital

,기관ID,기관명,경도,위도,응급실,수술실,입원실,병상수,일반중환자실,agglo_cluster,중심거리
0,A1100008,(학)고려대학교의과대학부속병원(안암병원),127.026471,37.587156,32,22,837,1048,20,0,3.815959
1,A1100001,경희대학교병원,127.051832,37.593877,21,16,604,850,20,0,1.683468
2,A1100048,노원을지대학교병원,127.070003,37.636443,23,10,449,512,0,0,3.388025
3,A1100044,녹색병원,127.086055,37.583621,10,3,299,318,15,0,3.316770
4,A1100021,삼육서울병원,127.065329,37.587992,21,5,406,431,17,0,2.092368
5,A1100050,서울성심병원,127.049838,37.584191,6,6,213,224,8,0,2.720431
6,A1100022,서울특별시동부병원,127.031403,37.575399,10,3,192,202,9,0,4.404144
7,A1100035,서울특별시서울의료원,127.098091,37.612869,22,9,549,623,None,0,3.244844
8,A1100075,의료법인풍산의료재단동부제일병원,127.109029,37.600676,7,2,104,104,0,0,4.186643
9,A1100020,의료법인한전의료재단한일병원,127.029024,37.646116,20,8,345,449,25,0,5.271497


In [ ]:
def hospi_map(start_name, goal_name):
    start_lon = float(hospital.loc[hospital['기관명'] == start_name, '경도'].values)
    start_lat = float(hospital.loc[hospital['기관명'] == start_name, '위도'].values)

    goal_lon = float(hospital.loc[hospital['기관명'] == goal_name, '경도'].values)
    goal_lat = float(hospital.loc[hospital['기관명'] == goal_name, '위도'].values)

    return start_lon, start_lat, goal_lon, goal_lat

start_lon, start_lat, goal_lon, goal_lat = hospi_map('서울동부혈액원', '서울특별시동부병원')

start_grid_5 = mg5.mapToGrid(start_lat, start_lon)
goal_grid_5 = mg5.mapToGrid(goal_lat, goal_lon)

start_grid_1 = mg1.mapToGrid(start_lat, start_lon)
goal_grid_1 = mg1.mapToGrid(goal_lat, goal_lon)

## # 기상 데이터
---

In [ ]:
Service_key = 'dDUc9SVR/pZIxs5qbu7ABEG4/S5t0ZJBTLZdlveVsR4Hyc2ynwn17cF68xglOrHN/mNu/W4JmkPP3N/M8HF12g==' # 서비스키 1
# Service_key = 'p6rQtYpFx5ZivDokXiWeZEPT1qg7CGrnWM9XP9rPcOy45BUhyI3RM9OoXCo3CWSAH31Va67TZGHaOOFWN/I7Cg==' # 서비스키 2

x = f'{start_grid_5[0]}' 
y = f'{start_grid_5[1]}'

def weather_forecast(Service_key, x, y):
    import requests
    import re
    import numpy as np 
    import pandas as pd
    from datetime import date, datetime, timezone, timedelta

    KST = timezone(timedelta(hours=9))
    time_record = datetime.now(KST)
    day = str(time_record)[:10]

    if time_record.time().minute > 40:
        time = str((time_record).time())[:2] + '00' # 실황에 사용
    elif time_record.time().minute <= 40:
        time = str((time_record - timedelta(hours=1)).time())[:2] + '00' # 예보에 사용

    # url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst' # 단기예보
    # url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst' # 초단기예보
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황
    params = {'serviceKey': f'{Service_key}',
              'pageNO': '1',
              'numOfRows': '60',
              'dataType': 'json',
              'base_date': f"{re.sub('-', '', day)}",
              'base_time': f'{time}',
              'nx': f'{x}',
              'ny': f'{y}'}

    response = requests.get(url, params=params).json()
    weather = response

    weather_df = pd.DataFrame(data = weather['response']['body']['items']['item'], index = ['강수형태', '습도', '1시간 강수량', '온도', '동서바람풍속', '동서바람풍향', '남북바람풍속', '남북바람풍향'])
    weather_df.drop(['baseDate', 'baseTime', 'nx', 'ny', 'category'], axis = 1, inplace = True)
    weather_df = weather_df.T

    pty_code = {0: '없음', 1: '비', 2: '비/눈', 3: '눈', 5: '빗방울', 6: '빗방울눈날림', 7: '눈날림'}
    pty_num = int(weather_df['강수형태']['obsrValue'])
    weather_df['강수형태']['obsrValue'] = pty_code[pty_num]

    if float(weather_df['동서바람풍속']['obsrValue']) > 0:
        weather_df['동서바람풍향']['obsrValue'] = '서'
    elif float(weather_df['동서바람풍속']['obsrValue']) < 0:
        weather_df['동서바람풍향']['obsrValue'] = '동'
    elif float(weather_df['동서바람풍속']['obsrValue']) == 0:
        weather_df['동서바람풍향']['obsrValue'] = '무'

    if float(weather_df['남북바람풍속']['obsrValue']) > 0:
        weather_df['남북바람풍향']['obsrValue'] = '남'
    elif float(weather_df['남북바람풍속']['obsrValue']) < 0:
        weather_df['남북바람풍향']['obsrValue'] = '북'
    elif float(weather_df['남북바람풍속']['obsrValue']) == 0:
        weather_df['남북바람풍향']['obsrValue'] = '무'

    # weather_df['동서바람풍속']['obsrValue'] = abs(float(weather_df['동서바람풍속']['obsrValue']))
    # weather_df['남북바람풍속']['obsrValue'] = abs(float(weather_df['남북바람풍속']['obsrValue']))

    return weather_df

weather_df = weather_forecast(Service_key, x, y)
weather_df

JSONDecodeError: ignored

# # 지도 행렬화
---

In [ ]:
start_grid_1, goal_grid_1

((302, 638), (300, 631))

In [ ]:
start_grid_1, goal_grid_1

lan_grid = [start_grid_1[0], goal_grid_1[0]]
lon_grid = [start_grid_1[1], goal_grid_1[1]]

lan_grid, lon_grid

([302, 300], [638, 631])

In [ ]:
# x, y 그리드를 뒤집어서 좌표를 만든다.
start_grid_dqn = [start_grid_1[1]-min(lon_grid)+1, start_grid_1[0]-min(lan_grid)+1]
goal_grid_dqn = [goal_grid_1[1]-min(lon_grid)+1, goal_grid_1[0]-min(lan_grid)+1]

start_grid_dqn, goal_grid_dqn

([7, 2], [0, 0])

In [ ]:
gridmap = []
for i in range(min(lan_grid), max(lan_grid)+1):
    for j in range(min(lon_grid), max(lon_grid)+1):
        gridmap.append([i, j])

In [ ]:
# 풍향 정하기

# 동서풍 (경도방향 길이가 길 경우)
if max(lan_grid) - min(lan_grid) >= max(lon_grid) - min(lon_grid):
    
    # 풍속으로 행렬 만들기
    maze_temp = []
    for grid in gridmap:
        map_temp = mg1.gridToMap(grid[0], grid[1])
        grid_temp = mg5.mapToGrid(map_temp[0], map_temp[1])
        weather_df = weather_forecast(Service_key, f'{grid_temp[0]}', f'{grid_temp[1]}')
        maze_temp.append(float(weather_df['동서바람풍속'].values))

        # 순풍 방향 정하기
        # 동풍
        if start_grid_1[0] >= goal_grid_1[0]:
            if -4 < float(weather_df['남북바람풍속'].values) <= 0:
                wind = 1
            elif -8 < float(weather_df['남북바람풍속'].values) <= -4:
                wind = 2
            elif -12 < float(weather_df['남북바람풍속'].values) <= -8:
                wind = 3
            elif 0 < float(weather_df['남북바람풍속'].values) <= 4:
                wind = 4
            elif 4 < float(weather_df['남북바람풍속'].values) <= 8:
                wind = 5
            elif 8 < float(weather_df['남북바람풍속'].values) <= 11:
                wind = 6
            else:
                wind = 0

        # 서풍
        elif start_grid_1[0] < goal_grid_1[0]:
            if 0 <= float(weather_df['남북바람풍속'].values) < 4:
                wind = 1
            elif 4 <= float(weather_df['남북바람풍속'].values) < 8:
                wind = 2
            elif 8 <= float(weather_df['남북바람풍속'].values) < 12:
                wind = 3
            elif -4 <= float(weather_df['남북바람풍속'].values) < 0:
                wind = 4
            elif -8 <= float(weather_df['남북바람풍속'].values) < -4:
                wind = 5
            elif -11 <= float(weather_df['남북바람풍속'].values) < -8:
                wind = 6
            else:
                wind = 0
            
        maze_temp.append(wind)


# 남북풍 (위도방향 길이가 길 경우)
if max(lon_grid) - min(lon_grid) > max(lan_grid) - min(lan_grid):
    
    # 풍속으로 행렬 만들기
    maze_temp = []
    for grid in gridmap:
        map_temp = mg1.gridToMap(grid[0], grid[1])
        grid_temp = mg5.mapToGrid(map_temp[0], map_temp[1])
        weather_df = weather_forecast(Service_key, f'{grid_temp[0]}', f'{grid_temp[1]}')

        # 순풍 방향 정하기
        # 북풍
        if start_grid_1[1] >= goal_grid_1[1]:
            if -4 < float(weather_df['남북바람풍속'].values) <= 0:
                wind = 1
            elif -8 < float(weather_df['남북바람풍속'].values) <= -4:
                wind = 2
            elif -12 < float(weather_df['남북바람풍속'].values) <= -8:
                wind = 3
            elif 0 < float(weather_df['남북바람풍속'].values) <= 4:
                wind = 4
            elif 4 < float(weather_df['남북바람풍속'].values) <= 8:
                wind = 5
            elif 8 < float(weather_df['남북바람풍속'].values) <= 11:
                wind = 6
            else:
                wind = 0

        # 남풍
        elif start_grid_1[1] < goal_grid_1[1]:
            if 0 <= float(weather_df['남북바람풍속'].values) < 4:
                wind = 1
            elif 4 <= float(weather_df['남북바람풍속'].values) < 8:
                wind = 2
            elif 8 <= float(weather_df['남북바람풍속'].values) < 12:
                wind = 3
            elif -4 <= float(weather_df['남북바람풍속'].values) < 0:
                wind = 4
            elif -8 <= float(weather_df['남북바람풍속'].values) < -4:
                wind = 5
            elif -11 <= float(weather_df['남북바람풍속'].values) < -8:
                wind = 6
            else:
                wind = 0
            
        maze_temp.append(wind)

In [ ]:
maze = np.array(maze_temp).reshape(max(lan_grid) - min(lan_grid) + 1, max(lon_grid) - min(lon_grid) + 1).T
maze = np.pad(maze, (1, 10))[:10, :10]
maze

NameError: ignored

# # 모든 병원 경로 추출
---

In [ ]:
import pymysql
import pandas as pd

conn = pymysql.connect(host='34.64.132.212', user='root', password='ASewetsvc124~1242#%1wsdeWXV', db='source')
sql_state='SELECT * FROM `군집화된병원6`'
hospital=pd.read_sql_query(sql_state, conn)

hospital

,기관ID,기관명,경도,위도,응급실,수술실,입원실,병상수,일반중환자실,agglo_cluster,중심거리
0,A1100008,(학)고려대학교의과대학부속병원(안암병원),127.026471,37.587156,32,22,837,1048,20,0,3.815959
1,A1100001,경희대학교병원,127.051832,37.593877,21,16,604,850,20,0,1.683468
2,A1100048,노원을지대학교병원,127.070003,37.636443,23,10,449,512,0,0,3.388025
3,A1100044,녹색병원,127.086055,37.583621,10,3,299,318,15,0,3.316770
4,A1100021,삼육서울병원,127.065329,37.587992,21,5,406,431,17,0,2.092368
5,A1100050,서울성심병원,127.049838,37.584191,6,6,213,224,8,0,2.720431
6,A1100022,서울특별시동부병원,127.031403,37.575399,10,3,192,202,9,0,4.404144
7,A1100035,서울특별시서울의료원,127.098091,37.612869,22,9,549,623,None,0,3.244844
8,A1100075,의료법인풍산의료재단동부제일병원,127.109029,37.600676,7,2,104,104,0,0,4.186643
9,A1100020,의료법인한전의료재단한일병원,127.029024,37.646116,20,8,345,449,25,0,5.271497


In [ ]:
import pymysql
import pandas as pd

conn = pymysql.connect(host='34.64.132.212', user='root', password='ASewetsvc124~1242#%1wsdeWXV', db='source')
sql_state='SELECT * FROM `드론공항위치`'
bloodbank=pd.read_sql_query(sql_state, conn)

bloodbank

,기관명,경도,위도,agglo_cluster
0,서울동부혈액원,127.061883,37.647144,0
1,재단법인아산사회복지재단서울아산병원,127.108238,37.526564,1
2,서울특별시보라매병원,126.924049,37.493718,2
3,세란병원,126.957707,37.575340,3
4,서울남부혈액원,127.048767,37.482129,4
5,서울중앙혈액원,126.870808,37.548049,5


In [ ]:
class map_grid():

    def __init__(self, grid = 5):
        import math

        self.NX = 149            ## X축 격자점 수
        self.NY = 253            ## Y축 격자점 수
        self.Re = 6371.00877     ##  지도반경
        self.grid = grid         ##  격자간격 (km)
        self.slat1 = 30.0        ##  표준위도 1
        self.slat2 = 60.0        ##  표준위도 2
        self.olon = 126.0        ##  기준점 경도
        self.olat = 38.0         ##  기준점 위도
        self.xo = 210 / self.grid     ##  기준점 X좌표
        self.yo = 675 / self.grid     ##  기준점 Y좌표
        self.first = 0

        if self.first == 0 :
            self.PI = math.asin(1.0) * 2.0
            self.DEGRAD = self.PI/ 180.0
            self.RADDEG = 180.0 / self.PI

            self.re = self.Re / self.grid
            self.slat1 = self.slat1 * self.DEGRAD
            self.slat2 = self.slat2 * self.DEGRAD
            self.olon = self.olon * self.DEGRAD
            self.olat = self.olat * self.DEGRAD

            self.sn = math.tan(self.PI * 0.25 + self.slat2 * 0.5) / math.tan(self.PI * 0.25 + self.slat1 * 0.5)
            self.sn = math.log(math.cos(self.slat1) / math.cos(self.slat2)) / math.log(self.sn)
            self.sf = math.tan(self.PI * 0.25 + self.slat1 * 0.5)
            self.sf = math.pow(self.sf, self.sn) * math.cos(self.slat1) / self.sn
            self.ro = math.tan(self.PI * 0.25 + self.olat * 0.5)
            self.ro = self.re * self.sf / math.pow(self.ro, self.sn)
            self.first = 1

    def mapToGrid(self, lat, lon, code = 0):
        import math

        ra = math.tan(self.PI * 0.25 + lat * self.DEGRAD * 0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI :
            theta -= 2.0 * self.PI
        if theta < -self.PI :
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        x = int(x + 1.5)
        y = int(y + 1.5)
        return x, y

    def gridToMap(self, x, y, code = 1):
        import math

        x = x - 1
        y = y - 1
        xn = x - self.xo
        yn = self.ro - y + self.yo
        ra = math.sqrt(xn * xn + yn * yn)
        if self.sn < 0.0 :
            ra = -ra
        alat = math.pow((self.re * self.sf / ra), (1.0 / self.sn))
        alat = 2.0 * math.atan(alat) - self.PI * 0.5
        if math.fabs(xn) <= 0.0 :
            theta = 0.0
        else :
            if math.fabs(yn) <= 0.0 :
                theta = self.PI * 0.5
                if xn < 0.0 :
                    theta = -theta
            else :
                theta = math.atan2(xn, yn)
        alon = theta / self.sn + self.olon
        lat = alat * self.RADDEG
        lon = alon * self.RADDEG

        return lat, lon



def make_maze(start_name, goal_name):

    import requests
    import re
    import numpy as np 
    import pandas as pd
    from datetime import date, datetime, timezone, timedelta


    # 위도, 경도 추출
    global hospital

    mg1 = map_grid(grid = 1)
    mg5 = map_grid(grid = 5)

    start_lon = float(hospital.loc[hospital['기관명'] == start_name, '경도'].values)
    start_lat = float(hospital.loc[hospital['기관명'] == start_name, '위도'].values)

    goal_lon = float(hospital.loc[hospital['기관명'] == goal_name, '경도'].values)
    goal_lat = float(hospital.loc[hospital['기관명'] == goal_name, '위도'].values)

    start_grid_5 = mg5.mapToGrid(start_lat, start_lon)
    goal_grid_5 = mg5.mapToGrid(goal_lat, goal_lon)

    start_grid_1 = mg1.mapToGrid(start_lat, start_lon)
    goal_grid_1 = mg1.mapToGrid(goal_lat, goal_lon)


    # 위도, 경도에서 기상 데이터 추출
    Service_key = 'dDUc9SVR/pZIxs5qbu7ABEG4/S5t0ZJBTLZdlveVsR4Hyc2ynwn17cF68xglOrHN/mNu/W4JmkPP3N/M8HF12g==' # 서비스키 1

    KST = timezone(timedelta(hours=9))
    time_record = datetime.now(KST)
    day = str(time_record)[:10]

    if time_record.time().minute > 40:
        time = str((time_record).time())[:2] + '00' # 실황에 사용
    elif time_record.time().minute <= 40:
        time = str((time_record - timedelta(hours=1)).time())[:2] + '00' # 예보에 사용

    # url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst' # 단기예보
    # url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst' # 초단기예보
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황
    params = {'serviceKey': f'{Service_key}',
              'pageNO': '1',
              'numOfRows': '60',
              'dataType': 'json',
              'base_date': f"{re.sub('-', '', day)}",
              'base_time': f'{time}',
              'nx': f'{x}',
              'ny': f'{y}'}

    response = requests.get(url, params=params).json()
    weather = response

    weather_df = pd.DataFrame(data = weather['response']['body']['items']['item'], index = ['강수형태', '습도', '1시간 강수량', '온도', '동서바람풍속', '동서바람풍향', '남북바람풍속', '남북바람풍향'])
    weather_df.drop(['baseDate', 'baseTime', 'nx', 'ny', 'category'], axis = 1, inplace = True)
    weather_df = weather_df.T

    pty_code = {0: '없음', 1: '비', 2: '비/눈', 3: '눈', 5: '빗방울', 6: '빗방울눈날림', 7: '눈날림'}
    pty_num = int(weather_df['강수형태']['obsrValue'])
    weather_df['강수형태']['obsrValue'] = pty_code[pty_num]

    if float(weather_df['동서바람풍속']['obsrValue']) > 0:
        weather_df['동서바람풍향']['obsrValue'] = '서'
    elif float(weather_df['동서바람풍속']['obsrValue']) < 0:
        weather_df['동서바람풍향']['obsrValue'] = '동'
    elif float(weather_df['동서바람풍속']['obsrValue']) == 0:
        weather_df['동서바람풍향']['obsrValue'] = '무'

    if float(weather_df['남북바람풍속']['obsrValue']) > 0:
        weather_df['남북바람풍향']['obsrValue'] = '남'
    elif float(weather_df['남북바람풍속']['obsrValue']) < 0:
        weather_df['남북바람풍향']['obsrValue'] = '북'
    elif float(weather_df['남북바람풍속']['obsrValue']) == 0:
        weather_df['남북바람풍향']['obsrValue'] = '무'


    # 행렬 만들기
    lan_grid = [start_grid_1[0], goal_grid_1[0]]
    lon_grid = [start_grid_1[1], goal_grid_1[1]]

    start_grid_dqn = [start_grid_1[1]-min(lon_grid)+1, start_grid_1[0]-min(lan_grid)+1]
    goal_grid_dqn = [goal_grid_1[1]-min(lon_grid)+1, goal_grid_1[0]-min(lan_grid)+1]

    gridmap = []
    for i in range(min(lan_grid), max(lan_grid)+1):
        for j in range(min(lon_grid), max(lon_grid)+1):
            gridmap.append([i, j])


    # 풍향 정하기
    # 동서풍 (경도방향 길이가 길 경우)
    maze_temp = []
    if max(lan_grid) - min(lan_grid) >= max(lon_grid) - min(lon_grid):
        
        # 풍속으로 행렬 만들기
        for grid in gridmap:
            map_temp = mg1.gridToMap(grid[0], grid[1])
            grid_temp = mg5.mapToGrid(map_temp[0], map_temp[1])
            weather_df = weather_forecast(Service_key, f'{grid_temp[0]}', f'{grid_temp[1]}')

            # 순풍 방향 정하기
            # 동풍
            if start_grid_1[0] >= goal_grid_1[0]:
                if -4 < float(weather_df['남북바람풍속'].values) <= 0:
                    wind = 1
                elif -8 < float(weather_df['남북바람풍속'].values) <= -4:
                    wind = 2
                elif -12 < float(weather_df['남북바람풍속'].values) <= -8:
                    wind = 3
                elif 0 < float(weather_df['남북바람풍속'].values) <= 4:
                    wind = 4
                elif 4 < float(weather_df['남북바람풍속'].values) <= 8:
                    wind = 5
                elif 8 < float(weather_df['남북바람풍속'].values) <= 11:
                    wind = 6
                else:
                    wind = 0                        
                # maze_temp.append(wind)


            # 서풍
            elif start_grid_1[0] < goal_grid_1[0]:
                if 0 <= float(weather_df['남북바람풍속'].values) < 4:
                    wind = 1
                elif 4 <= float(weather_df['남북바람풍속'].values) < 8:
                    wind = 2
                elif 8 <= float(weather_df['남북바람풍속'].values) < 12:
                    wind = 3
                elif -4 <= float(weather_df['남북바람풍속'].values) < 0:
                    wind = 4
                elif -8 <= float(weather_df['남북바람풍속'].values) < -4:
                    wind = 5
                elif -11 <= float(weather_df['남북바람풍속'].values) < -8:
                    wind = 6
                else:
                    wind = 0      

            maze_temp.append(wind)


    # 남북풍 (위도방향 길이가 길 경우)
    elif max(lan_grid) - min(lan_grid) < max(lon_grid) - min(lon_grid):
        
        # 풍속으로 행렬 만들기
        for grid in gridmap:
            map_temp = mg1.gridToMap(grid[0], grid[1])
            grid_temp = mg5.mapToGrid(map_temp[0], map_temp[1])
            weather_df = weather_forecast(Service_key, f'{grid_temp[0]}', f'{grid_temp[1]}')

            # 순풍 방향 정하기
            # 북풍
            if start_grid_1[1] >= goal_grid_1[1]:
                if -4 < float(weather_df['남북바람풍속'].values) <= 0:
                    wind = 1
                elif -8 < float(weather_df['남북바람풍속'].values) <= -4:
                    wind = 2
                elif -12 < float(weather_df['남북바람풍속'].values) <= -8:
                    wind = 3
                elif 0 < float(weather_df['남북바람풍속'].values) <= 4:
                    wind = 4
                elif 4 < float(weather_df['남북바람풍속'].values) <= 8:
                    wind = 5
                elif 8 < float(weather_df['남북바람풍속'].values) <= 11:
                    wind = 6
                else:
                    wind = 0
                # maze_temp.append(wind)

            # 남풍
            elif start_grid_1[1] < goal_grid_1[1]:
                if 0 <= float(weather_df['남북바람풍속'].values) < 4:
                    wind = 1
                elif 4 <= float(weather_df['남북바람풍속'].values) < 8:
                    wind = 2
                elif 8 <= float(weather_df['남북바람풍속'].values) < 12:
                    wind = 3
                elif -4 <= float(weather_df['남북바람풍속'].values) < 0:
                    wind = 4
                elif -8 <= float(weather_df['남북바람풍속'].values) < -4:
                    wind = 5
                elif -11 <= float(weather_df['남북바람풍속'].values) < -8:
                    wind = 6
                else:
                    wind = 0

            maze_temp.append(wind)
                

    maze = np.array(maze_temp).reshape(max(lan_grid) - min(lan_grid) + 1, max(lon_grid) - min(lon_grid) + 1).T
    maze = np.pad(maze, (1, 10))[:12, :12]

    return maze, start_grid_dqn, goal_grid_dqn

In [ ]:
clu0_서울특별시동부병원, start_grid_dqn, goal_grid_dqn = make_maze(hospital, '서울동부혈액원', '서울특별시동부병원')

In [ ]:
clu0_서울특별시동부병원

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

## # 지도 저장
---

In [ ]:
for i in bloodbank_name:
    bloodbank_cluster = int(bloodbank.loc[bloodbank['기관명'] == i, 'agglo_cluster'])
    hospital_name = sorted(hospital.loc[hospital['agglo_cluster'] == bloodbank_cluster].drop(hospital.loc[hospital['기관명'] == i].index, axis = 0)['기관명'])

    for j in hospital_name:
        maze, start_grid_dqn, goal_grid_dqn = make_maze(hospital, i, j)
        np.savez(f'/content/drive/MyDrive/workspace/cakd5/3차_project/작업물/김기현/data/maze/clu{bloodbank_cluster}_{j}.npz',
                 maze = maze,
                 start_grid_dqn = start_grid_dqn,
                 goal_grid_dqn = goal_grid_dqn)

## # 지도 불러오기
---

In [ ]:
import os 
import numpy as np

path = '/content/drive/MyDrive/workspace/cakd5/3차_project/작업물/김기현/data/maze/'
maze_list = os.listdir(path)

for i, j in enumerate(maze_list):
    maze_info = np.load(path + j)
    maze = maze_info['maze']
    start_grid_dqn = tuple(maze_info['start_grid_dqn'])
    goal_grid_dqn = tuple(maze_info['goal_grid_dqn'])
    print(i, j)
    print(maze)
    print('start :', start_grid_dqn)
    print('goal :', goal_grid_dqn)
    print()

0 clu0_(학)고려대학교의과대학부속병원(안암병원).npz
[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
start : (7, 4)
goal : (1, 1)

1 clu0_경희대학교병원.npz
[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
start : (6, 2)
goal : (1, 1)

2 clu0_노원을지대학교병원.npz
[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 4 4 0 0 0 0 0 0 0 0 0]
 [0 4 4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 

In [ ]:
maze

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
       [0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
       [0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
       [0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
start_grid_dqn, goal_grid_dqn

((1, 5), (4, 1))

In [ ]:
# 기상 변수 무작위로 부여하기
col_grid = [start_grid_dqn[0], goal_grid_dqn[0]]
row_grid = [start_grid_dqn[1], goal_grid_dqn[1]]

# col_size = max(col_grid) - min(col_grid) + 1
# row_size = max(row_grid) - min(row_grid) + 1

col_size = abs(start_grid_dqn[0] - goal_grid_dqn[0]) + 1
row_size = abs(start_grid_dqn[1] - goal_grid_dqn[1]) + 1

p = [0.01, 0.3, 0.145, 0.05, 0.3, 0.145, 0.05]
random_maze = np.random.choice(np.arange(0, 7), col_size * row_size, p = p)
maze[min(col_grid) : max(col_grid)+1, min(row_grid) : max(row_grid)+1] = random_maze.reshape(col_size, row_size)
maze[start_grid_dqn] = 1; maze[goal_grid_dqn] = 1
maze

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 5, 4, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 4, 1, 4, 5, 0, 0, 0, 0, 0, 0],
       [0, 1, 5, 6, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 1, 4, 5, 5, 4, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
bloodbank_input = input(f'출발지 (아래 목록 중 택 1):\n{bloodbank["기관명"].values}\n')
print()
cluster = bloodbank.loc[bloodbank['기관명'] == bloodbank_input, 'agglo_cluster']
selected_hospital = hospital.loc[hospital['agglo_cluster'] == int(cluster), '기관명'].values

hospital_input = input(f'도착지 (아래 목록 중 택 1):\n{selected_hospital}\n')

출발지 (아래 목록 중 택 1):
['서울동부혈액원' '재단법인아산사회복지재단서울아산병원' '서울특별시보라매병원' '세란병원' '서울남부혈액원' '서울중앙혈액원']
서울동부혈액원

도착지 (아래 목록 중 택 1):
['(학)고려대학교의과대학부속병원(안암병원)' '경희대학교병원' '노원을지대학교병원' '녹색병원' '삼육서울병원' '서울성심병원'
 '서울특별시동부병원' '서울특별시서울의료원' '의료법인풍산의료재단동부제일병원' '의료법인한전의료재단한일병원' '인제대학교상계백병원'
 '한국원자력의학원원자력병원' '한양대학교병원' '서울동부혈액원']
한국원자력의학원원자력병원


In [ ]:
hospital.loc[hospital['agglo_cluster'] == int(cluster), '기관명'].values

array(['(학)고려대학교의과대학부속병원(안암병원)', '경희대학교병원', '노원을지대학교병원', '녹색병원', '삼육서울병원',
       '서울성심병원', '서울특별시동부병원', '서울특별시서울의료원', '의료법인풍산의료재단동부제일병원',
       '의료법인한전의료재단한일병원', '인제대학교상계백병원', '한국원자력의학원원자력병원', '한양대학교병원',
       '서울동부혈액원'], dtype=object)